In [ ]:
import os
import numpy as np
import torch
import cv2
import json
from CACNet import CACNet
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from io import BytesIO

In [ ]:
memory_model_size = torch.cuda.max_memory_allocated()/1024/1024
print(f'{memory_model_size}')


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE.type=='cuda'

In [ ]:
IMAGE_SIZE = (224,224)
IMAGE_NET_MEAN = [0.485, 0.456, 0.406]
IMAGE_NET_STD = [0.229, 0.224, 0.225]

class ClearCache:
    def __enter__(self):
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        torch.cuda.empty_cache()


class ImagesDataset(Dataset):
    def __init__(self, images_as_bytes:list[bytes]) :
        self.images_as_bytes = images_as_bytes 
        # self.resized_image,self.im_widths,self.im_heights  = self.process_images(images_as_bytes)  
        self.IMAGE_SIZE = torch.tensor(IMAGE_SIZE, requires_grad=False)
        self.image_transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=IMAGE_NET_MEAN, std=IMAGE_NET_STD)])

    def  process_images(self,image):
        image = Image.open(BytesIO(image)).convert('RGB')
        im_width, im_height = image.size
        h = self.IMAGE_SIZE[0]
        w = self.IMAGE_SIZE[1]
        resized_image = image.resize((w, h), Image.LANCZOS)
        im_widths = torch.tensor(im_widths, requires_grad=False)
        im_heights = torch.tensor(im_heights, requires_grad=False)
        resized_image = self.image_transformer(resized_image)
        resized_image.requires_grad=False
        return  resized_image, im_width, im_height  
    
    def __len__(self,):
        return len(self.images_as_bytes)
    
    def __getitem__(self,index):
        return self.process_images(self.images_as_bytes[index])


In [ ]:
import subprocess as sp
import os

def get_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    return memory_free_values

get_gpu_memory()[0]


In [ ]:
(get_gpu_memory()[0]*1024*1024)/(3*4*224*224*50)

In [ ]:
a=[1]
b=[2]
a.extend(b)
a

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
weight_file = "./pretrained_models/best-FLMS_iou.pth"
model = CACNet(loadweights=False)
model.load_state_dict(torch.load(weight_file,map_location=device))
model = model.to(device).eval()

In [ ]:
IMAGE_NET_MEAN = [0.485, 0.456, 0.406]
IMAGE_NET_STD = [0.229, 0.224, 0.225]
IMAGE_SIZE = (224,224)
def parse_annotation(index):
    data_file = os.path.join(cfg.FCDB_dir, 'cropping_testing_set.json')
    origin_data = json.loads(open(data_file, 'r').read())
    image_data = origin_data[index]
    image = os.path.split(image_data['url'])[-1]
    x,y,w,h = image_data['crop']
    gt_crop= [x,y,x+w,y+h]
    gt_crop = np.array(gt_crop).reshape(-1,4).astype(np.float32)
    return image,gt_crop
    
def process_image(image_file):
    image = Image.open(image_file).convert('RGB')
    im_width, im_height = image.size
    h = IMAGE_SIZE[1]
    w = IMAGE_SIZE[0]
    resized_image = image.resize((w, h), Image.LANCZOS)
    image_transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGE_NET_MEAN, std=IMAGE_NET_STD)])
    im = image_transformer(resized_image)
    return im, im_width, im_height, image_file

def predict(image,model = model):
    logits,kcm,crop = model(image, only_classify=False)
    return logits,kcm,crop

def crop_image(image):
    im, im_width, im_height, image_file = process_image(image)
    if im.dim()<4:
        im = im.unsqueeze(0)
    im = im.to(device)
    logits,kcm,crop = predict(im)
    print(im.shape)
    crop[:,0::2] = crop[:,0::2] / im.shape[-1] * im_width
    crop[:,1::2] = crop[:,1::2] / im.shape[-2] * im_height
    pred_crop = crop.detach().cpu()
    pred_crop[:,0::2] = torch.clip(pred_crop[:,0::2], min=0, max=im_width)
    pred_crop[:,1::2] = torch.clip(pred_crop[:,1::2], min=0, max=im_height)
    pred_crop = pred_crop[0].numpy().tolist()
    x1,y1,x2,y2 = [int(x) for x in pred_crop]
    source_img = cv2.imread(image_file)
    source_img = cv2.rectangle(source_img , (x1,y1), (x2,y2), (255,0,0), 2) 
    return source_img

In [ ]:

print(id(IMAGE_SIZE))

In [ ]:
image_file,gt_crop = parse_annotation(3)
x1,y1,x2,y2 = [int(x) for x in gt_crop[0]]
image_file = os.path.join(cfg.FCDB_dir,f'data/{image_file}')
image_file = crop_image(image_file)
image_file = cv2.rectangle(image_file , (x1,y1), (x2,y2), (0,255,0), 2) 
import matplotlib.pyplot as plt 
plt.imshow(image_file)
print(image_file.shape)

In [ ]:
from crop import Cropper
image_file,gt_crop = parse_annotation(3)
image_file = os.path.join(cfg.FCDB_dir,f'data/{image_file}')
crop_model = Cropper()
img,x1,y1,x2,y2 = crop_model.crop_image(image_file)
import matplotlib.pyplot as plt 
plt.imshow(img)

In [ ]:
from PIL import Image, ImageDraw 
import numpy as np
img = Image.open('sample.jpg' )
draw = ImageDraw.Draw(img.copy())
bbox = {
  "x1": 7,
  "y1": 38,
  "x2": 1019,
  "y2": 567
}
draw.rectangle(((bbox["x1"], bbox["y1"]),(bbox["x2"], bbox["y2"])), outline='Red')
cropped_image = img.crop(list(bbox.values()))
import matplotlib.pyplot as plt 
# print(np.array(img))
plt.imshow(cropped_image)

In [ ]:
import io
a=io.BytesIO()
np.save(a, img)
# print(a)
# b = np.load(io.BytesIO(a.getvalue()))
Image.open(a).convert('RGB')

print(a)